In [17]:
import requests
import json
import pandas as pd
import numpy as np

##나의 key
key = 'bf388499b71a365d725e1c888201736f7409d7e4'

In [18]:
start_year = '2016-03-01'
start_y = 2025
start_q = 1
export_import = 'expDlr'

dates_period = pd.date_range(start='2020-01', end='2025-05',freq='ME')

dates_list1 = []
for dates in dates_period:
    temp = str(dates)[:7]
    dates_list1.append(temp)

In [19]:
def get_us_export_data(hs_list, start='2013-01', end='2025-05', api_key='your_key_here'):
    """
    미국 HS 코드별 수출 데이터를 월별로 가져옵니다.

    Parameters:
        hs_list (list): 조회할 HS 코드 리스트
        start (str): 시작 날짜 (yyyy-mm)
        end (str): 종료 날짜 (yyyy-mm)
        api_key (str): U.S. Census API 키

    Returns:
        pd.DataFrame: 월별 수출 데이터
        pd.DataFrame: 분기별 수출 데이터
    """
    us_export_hs = []
    date_range = pd.date_range(start=start, end=end, freq='MS')  # 매월 시작일

    for hs in hs_list:
        for dt in date_range:
            year = dt.strftime('%Y')
            month = dt.strftime('%m')
            url = f"https://api.census.gov/data/timeseries/intltrade/exports/hs?get=ALL_VAL_MO&key={api_key}&YEAR={year}&MONTH={month}&E_COMMODITY={hs}"
            try:
                res = requests.get(url)
                if res.status_code == 200:
                    data = json.loads(res.text)
                    if len(data) > 1:
                        temp = data[1]
                        us_export_hs.append(temp)
                else:
                    print(f"❌ 실패: {year}-{month} {hs} → Status: {res.status_code}")
            except Exception as e:
                print(f"⚠️ 예외 발생: {year}-{month} {hs} → {e}")

    if not us_export_hs:
        print("❌ 가져온 데이터가 없습니다.")
        return None, None

    # 데이터프레임 생성 및 컬럼 설정
    df = pd.DataFrame(us_export_hs, columns=['expDlr', 'year', 'month', 'hs_code'])

    # 숫자가 아닌 값은 NaN 처리하고 float으로 변환
    df['expDlr'] = pd.to_numeric(df['expDlr'], errors='coerce')

    # 너무 큰 값은 비정상으로 간주하고 NaN 처리
    df.loc[df['expDlr'] > 1e18, 'expDlr'] = np.nan

    # 날짜 처리 및 인덱스 설정
    df['Date'] = pd.to_datetime(df['year'] + '-' + df['month'], errors='coerce')
    df.dropna(subset=['Date'], inplace=True)
    df.set_index('Date', inplace=True)
    df['quarter'] = df.index.to_period('Q')

    # 월별 및 분기별 데이터 생성
    df_monthly = df.copy()
    df_quarterly = df.groupby(['quarter', 'hs_code'])['expDlr'].sum().reset_index()
    df_quarterly['quarter'] = df_quarterly['quarter'].dt.to_timestamp()

    return df_monthly, df_quarterly

In [20]:
hs_codes = ['854231', '854232']
us_export_month, us_export_quarter = get_us_export_data(
    hs_list=hs_codes,
    start='2013-01',
    end='2025-04',
    api_key= key  # 실제 API 키로 바꾸세요
)